#### Import necessary libs

In [30]:
from applications.graph.graph_problem import GraphProblem
from classical_to_quantum.applications.graph.grover_applications.graph_oracle import *
from classical_to_quantum.algorithms.grover import GroverWrapper
from classical_to_quantum.applications.graph.grover_applications.graph_color import GraphColor



### Example: Independent Set

In [31]:
graph_problem = GraphProblem("/Users/mac/workspace/quantum-journey/QUANTUM-CLASSICAL-TRANSLATION/classical_to_quantum/cases/Gset/G0")
independent_set_cnf = independent_set_to_sat(graph_problem.graph())
independent_set_oracle = cnf_to_quantum_oracle(independent_set_cnf)
def fun(state):
    return True
grover = GroverWrapper(oracle=independent_set_oracle,
                       iterations=2,
                       is_good_state=fun,
                       objective_qubits=list(range(graph_problem.num_nodes)))
res = grover.run(verbose=True)
top_measurements = res.top_measurement
print(top_measurements)

10
     ┌───┐┌────┐┌────┐
q_0: ┤ H ├┤0   ├┤0   ├
     ├───┤│    ││    │
q_1: ┤ H ├┤1   ├┤1   ├
     ├───┤│    ││    │
q_2: ┤ H ├┤2   ├┤2   ├
     ├───┤│    ││    │
q_3: ┤ H ├┤3   ├┤3   ├
     └───┘│    ││    │
q_4: ─────┤4   ├┤4   ├
          │  Q ││  Q │
q_5: ─────┤5   ├┤5   ├
          │    ││    │
q_6: ─────┤6   ├┤6   ├
          │    ││    │
q_7: ─────┤7   ├┤7   ├
          │    ││    │
q_8: ─────┤8   ├┤8   ├
          │    ││    │
q_9: ─────┤9   ├┤9   ├
          └────┘└────┘
c: 4/═════════════════
                      
{   'assignment': '0000',
    'circuit_results': [   {   '0000': 0.1406249999999966,
                               '0001': 0.1406249999999965,
                               '0010': 0.1406249999999962,
                               '0011': 0.0156249999999996,
                               '0100': 0.1406249999999961,
                               '0101': 0.0156249999999996,
                               '0110': 0.0156249999999996,
                             

### Example: graph coloring

In [36]:
coloring_problem = GraphColor("/Users/mac/workspace/quantum-journey/QUANTUM-CLASSICAL-TRANSLATION/classical_to_quantum/cases/Gset/G0", verbose=True)
res = coloring_problem.search()
res

Variable Qubits: [0, 1, 2, 3, 4, 5, 6, 7]
Check Qubits: [8, 9, 10, 11, 12]
Disagree List: [[[0, 1], [2, 3]], [[0, 1], [4, 5]], [[2, 3], [4, 5]], [[2, 3], [6, 7]], [[4, 5], [6, 7]]]
Output Qubit: 13
14
      ┌───┐┌─────┐
 q_0: ┤ H ├┤0    ├
      ├───┤│     │
 q_1: ┤ H ├┤1    ├
      ├───┤│     │
 q_2: ┤ H ├┤2    ├
      ├───┤│     │
 q_3: ┤ H ├┤3    ├
      ├───┤│     │
 q_4: ┤ H ├┤4    ├
      ├───┤│     │
 q_5: ┤ H ├┤5    ├
      ├───┤│     │
 q_6: ┤ H ├┤6    ├
      ├───┤│   Q │
 q_7: ┤ H ├┤7    ├
      └───┘│     │
 q_8: ─────┤8    ├
           │     │
 q_9: ─────┤9    ├
           │     │
q_10: ─────┤10   ├
           │     │
q_11: ─────┤11   ├
           │     │
q_12: ─────┤12   ├
           │     │
q_13: ─────┤13   ├
           └─────┘
 c: 8/════════════
                  
